# Spectrogram Notebook (PRN Overlay)

This notebook loads processed IQ data, computes a short-time Fourier transform, and overlays Doppler estimates parsed from GPS-SDR-Receiver logs.

$$
X[n] \in \mathbb{C},\;\text{STFT}(X)(f,t)=\sum_{m} X[m] w[m-t] e^{-j2\pi fm/ f_s}.\\
\text{Display } 20\log_{10}|\text{STFT}(X)|,\; f\in[-f_s/2,f_s/2],\; t\text{ windows}.\\
\text{Overlay doppler lines } f_{\text{doppler}} = \Delta f_{\text{log}}.
$$

In [2]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from scipy import signal
import re
plt.style.use(['science','grid'])

In [3]:
DATA_PATH = Path('../data/processed/TGS_L1_E1_2p048Msps_iq_u8.bin')
LOG_PATH = Path('../logs/run_fairdata.log')
FS_HZ = 2_048_000
WINDOW_SAMPLES = FS_HZ * 2  # analyze first 2 seconds
N_PER_SEG = 8192
N_OVERLAP = 6144

In [4]:
PRN_PATTERN = re.compile(r"PRN\s+(\d+)\s+Corr:([0-9.]+)\s+f=([+\-0-9.]+)")

def load_iq(path: Path, max_samples: int) -> np.ndarray:
    data = np.fromfile(path, dtype=np.uint8, count=2 * max_samples)
    if data.size < 2:
        raise RuntimeError('File is empty or shorter than requested window')
    data = (data.astype(np.float32) - 128.0) / 128.0
    i = data[0::2]
    q = data[1::2]
    return (i + 1j * q)[:max_samples]


def parse_prn_log(path: Path) -> dict[int, dict[str, float]]:
    info = {}
    if not path.exists():
        return info
    with path.open() as fh:
        for line in fh:
            match = PRN_PATTERN.search(line)
            if match:
                prn = int(match.group(1))
                info[prn] = {
                    'corr': float(match.group(2)),
                    'doppler_hz': float(match.group(3)),
                }
    return info

In [ ]:
iq = load_iq(DATA_PATH, WINDOW_SAMPLES)
prn_info = parse_prn_log(LOG_PATH)
print(f'Loaded {iq.size} IQ samples (first {WINDOW_SAMPLES})')
print('PRNs from log:', prn_info)


Loaded 4096000 IQ samples (first 4096000)
PRNs from log: {}


: 

In [ ]:
f, t, Zxx = signal.stft(iq, fs=FS_HZ, nperseg=N_PER_SEG, noverlap=N_OVERLAP, boundary=None, padded=False, return_onesided=False)
power = 20.0 * np.log10(np.abs(Zxx) + 1e-12)
f_shift = np.fft.fftshift(f) / 1e3  # kHz
t_seconds = t
power_shift = np.fft.fftshift(power, axes=0)
fig, ax = plt.subplots(figsize=(10, 5))
mesh = ax.pcolormesh(t_seconds, f_shift, power_shift, shading='gouraud', cmap='magma')
for prn, meta in prn_info.items():
    ax.axhline(meta['doppler_hz'] / 1e3, linestyle='--', label=f"PRN {prn}")
ax.set_xlabel('Time (s)')
ax.set_ylabel('Frequency (kHz)')
ax.set_title('STFT magnitude (dB) with Doppler overlays')
if prn_info:
    ax.legend(loc='upper right', fontsize='small')
fig.colorbar(mesh, ax=ax, label='Magnitude (dB)')
fig.tight_layout()
